In [ ]:
import pandas as pd
import os 

In [ ]:
downloads_23andME = '/Users/jerenolsen/Desktop/genome_downloads/23andMe_final_dir'
#702 files originally

In [ ]:
def get_invalid_files(folder):

    headers = []
    failed_files = []
    filepaths = get_file_paths(folder)
    
    #Try reading 
    for path in filepaths:
        header = read_header(path)
        if header:
            headers.append(header)
        else:
            failed_files.append(path.split('/')[-1])
            
    for header in headers:
        #Check for 23andMe header
        if not check_23andMe(header):
            failed_files.append(header[0])
            
        #Check for build
        if not check_build(header):
            failed_files.append(header[0])
            
        #Check Columns
        if not check_columns(header):
            failed_files.append(header[0])
        
    failed_files = list(set(failed_files))
    passed_headers = [header for header in headers if header[0] not in failed_files]
            
    return passed_headers, failed_files
    

In [ ]:
def read_header(path_23andMe):
    header = []
    with open(path_23andMe, 'r') as f:
        header.append(path_23andMe.split('/')[-1])
        try:
            for line in f:
                if line[0] == '#':
                    header.append(line)
                else:
                    break
        except Exception as e:
            return False
                
    return header

def get_file_paths(folder):
    paths = [path for path in os.listdir(folder)]
    paths = [os.path.join(folder, path) for path in paths]
    return paths

def check_23andMe(header):
    try:
        if len(header) == 1 or 'generated by 23andMe' not in header[1]:
            return False
    except Exception as e:
        return False
    
    return True

def check_build(header):
    build_lines = [line for line in header if 'build' in line]
    build_lines = [line for line in build_lines if 'we are using' in line.lower()]
    if build_lines == []:
        return False
    return True

def check_columns(header):
    cols = header[-1]
    if 'rsid' not in cols:
        return False
    return True


In [ ]:
passed_headers, invalid_files = get_invalid_files(downloads_23andME)

In [ ]:
def remove_invalid_files(file_dir, invalid_files):
    for file in invalid_files:
        path = os.path.join(file_dir,file)
        os.remove(path)
        
    return True
        

In [ ]:
remove_invalid_files(downloads_23andME, invalid_files)